In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

# The setup

Imagine you have a research question, a dataset, and a novel model that you are sure will do better. 
How do you robustly compare it to any baseline you have, so to have convincing results to present?

Let us create some mock data to represent the problem, and analyze the use of statistical tests and confidence intervals within this context

## Simulating an experiment

We have a dataset, two baselines (B1 and B2), and our proposed model (M).

We split the dataset into training and test set, train the model, and measure the Mean Squared Error (MSE).
The measure of the performance of a model is a random variable, and we will skip here any form of training and simply sample
from some underlying distribution to have some results to play with.

In [2]:
np.random.seed(42)

results = []
results.append({"Model": "B1", "MSE": np.random.normal(loc=0.87, scale=0.021)})
results.append({"Model": "B2", "MSE": np.random.normal(loc=0.85, scale=0.034)})
results.append({"Model": "M", "MSE": np.random.normal(loc=0.79, scale=0.10)})
results = pd.DataFrame(results)
results

,Model,MSE
0,B1,0.880431
1,B2,0.845299
2,M,0.854769


Notice that if we repeat the procedure, we will get different results, that might even flip any ranking that we derive. 
This is not just an artifact of this notebook, but a real fact of research projects: if you repeat the same experiment with a different
train/test split, or with a different initialization of parameters, or possibly even just a different seed, you will get a different metric.

In [3]:
results = []
results.append({"Model": "B1", "MSE": np.random.normal(loc=0.87, scale=0.021)})
results.append({"Model": "B2", "MSE": np.random.normal(loc=0.85, scale=0.034)})
results.append({"Model": "M", "MSE": np.random.normal(loc=0.79, scale=0.10)})
results = pd.DataFrame(results)
results

,Model,MSE
0,B1,0.901984
1,B2,0.842039
2,M,0.766586


## Repeated evaluations

To solve this issue, we need to have multiple measurements of a model's performance, so that we may draw conclusions about their distribution.
One way to achieve that is with a method like [nested cross-validation](https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/), or by sampling multiple train/test splits. Using multiple training runs with the same train/test split is not recommended, as it is overly biased to this specific data split.

Let us assume that we have performed 5-fold nested cross-validation, with the following results.

In [4]:
results = []
for split_idx in range(5):
    results.append({"Model": "B1", "Split": split_idx, "MSE": np.random.normal(loc=0.87, scale=0.021)})
    results.append({"Model": "B2", "Split": split_idx, "MSE": np.random.normal(loc=0.85, scale=0.034)})
    results.append({"Model": "M", "Split": split_idx, "MSE": np.random.normal(loc=0.79, scale=0.10)})
results = pd.DataFrame(results)
results

,Model,Split,MSE
0,B1,0,0.903163
1,B2,0,0.876093
2,M,0,0.743053
3,B1,1,0.881394
4,B2,1,0.834244
5,M,1,0.743427
6,B1,2,0.875081
7,B2,2,0.784948
8,M,2,0.617508
9,B1,3,0.858192


We can now summarize the performance of each model by averaging over the different splits. 
Thanks to the magic of the [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem), this method will provide a more representative estimate of the 
performance.

In [23]:
mean_df = results.drop("Split", axis=1).groupby("Model").mean()
mean_df

,MSE
Model,
B1,0.873752
B2,0.822566
M,0.772395


But how confident are we in these results? We could provide a measure of uncertainty using the [Standard Error of the Mean](https://en.wikipedia.org/wiki/Standard_error).
This approach is correct, but not quite interpretable.

In [25]:
std_err_df = results.drop("Split", axis=1).groupby("Model").std()/np.sqrt(5)
std_err_df.columns = ["MSE_stderr"]
std_err_df

,MSE_stderr
Model,
B1,0.009187
B2,0.015635
M,0.052462


In [27]:
summary_df = mean_df.join(std_err_df)
summary_df

,MSE,MSE_stderr
Model,,
B1,0.873752,0.009187
B2,0.822566,0.015635
M,0.772395,0.052462


## Calculating the confidence intervals

A much better alternative is the use of [Confidence Intervals](https://en.wikipedia.org/wiki/Confidence_interval), ranges of values that
contain the target metric (average performance), with a specified level of confidence. 

The process to obtain confidence intervals is the following:
- Specify what level of confidence you want to have (95% is a pretty common choice, meaning that with 95% probability the true value will be inside the interval).
- Identify the distribution of the target quantity
- Find the quantiles that reflect your desired confidence intervals

Let us see how this works in our example:

- We choose 95% as the standard CI level of confidence. 
- Our target metric is the average MSE. Being an average of a sample of 5 measurements, this quantity follows a [t-Distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution) with 4 Degrees of Freedom, mean $\mu$ (the expected value of the mean), and standard deviation equal to the standard error of the mean $\frac{\sigma}{\sqrt{n}}$, where $\sigma$ is the population standard deviation of the measurements.
- Using Maximum likelihood, we can substitute the observed values as estimates of the distribution parameters
- Given the way the standard t-Distribution can be transformed and scaled (analogous to the Gaussian distribution), we can calculate the necessary quantiles using the standard t-Distribution with the correct number of DoFs, and then shift and scale the results with the estimated values.
- So if $\bar{X}$ is the average MSE for a model, and $S$ is the standard deviation obtained from the $n=5$ measurements, the confidence interval that we are looking for is $\bar{X} \pm t_q * \frac{S}{\sqrt{n}}$. In this formula, $t_q$ is the selected quantile calculated using the standard t-Distribution, and we can use teh $\pm$ notation because the distribution is symmetrical with respect to the mean.
- Since we want to have 95% confidence intervals centered around the mean, we will use the quantiles (2.5%, 97.5%), which are symmetrical.

Using this setup, we obtain the following:


In [22]:
dof=4
t_q = scipy.stats.t.sf(0.975, dof)
t_q

np.float64(0.1923843459783338)

In [28]:
summary_df["Half_CI"] = summary_df["MSE_stderr"]*t_q
summary_df

,MSE,MSE_stderr,Half_CI
Model,,,
B1,0.873752,0.009187,0.001767
B2,0.822566,0.015635,0.003008
M,0.772395,0.052462,0.010093


In [30]:
for i, row in summary_df.iterrows():
    print(f"Model {i}: {row.MSE:.3f} ± {row.Half_CI:.3f}")

Model B1: 0.874 ± 0.002
Model B2: 0.823 ± 0.003
Model M: 0.772 ± 0.010


## What if my confidence intervals overlap?

CIs are a good representation of the performance, but you might want to convey a firm yes or no answer to the question "Is my model clearly better than the rest?".
To provide such an outcome, we can make use of suitable statistical tests, such as the [t-test](https://en.wikipedia.org/wiki/Student%27s_t-test).
The procedure goes as follows:
- Select a statistical significance threshold (0.05 is common, although I personally prefer to use 0.01).
- Take the model with the best average metric, and perform a pairwise t-test between it and each of the other models. If any of those tests fails to reject the null hypothesis at the chosen significance threshold, we cannot confidently claim that the performance difference of the two models is statistically significant, and they should be reported as comparable.

Since the variance of the samples for each model is not necessarily equal, we can make use of the [Welch's t-test](https://en.wikipedia.org/wiki/Welch%27s_t-test) rather than a simple t-test.

It is a matter of debate whether you should use some multiple hypothesis correction (such as the Bonferroni correction, or the Benjamini-Hochberg procedure) to this set of comparisons. If you want to err on the side of caution, you should include this step. 

With our results, the process is the following:

In [38]:
statistical_significance_threshold = 0.01
best_model_samples = results[results["Model"]=="M"]["MSE"].tolist()

for alternative_model in ["B1", "B2"]:
    alternative_model_samples = results[results["Model"]==alternative_model]["MSE"].tolist()
    ttest_result = scipy.stats.ttest_ind(best_model_samples, alternative_model_samples, equal_var=False)
    if ttest_result.pvalue<statistical_significance_threshold:
        print(f"Difference between model M and model {alternative_model} is statistically significant: pvalue {ttest_result.pvalue}")
    else:
        print(f"The difference between model M and model {alternative_model} is NOT statistically significant: pvalue {ttest_result.pvalue}")


The difference between model M and model B1 is NOT statistically significant: pvalue 0.12565754369221083
The difference between model M and model B2 is NOT statistically significant: pvalue 0.40394414226204217


We see that in this case the difference is not statistically significant when comparing to either baseline.
But how can that be? We can clearly see in the mock data creation that we used different distributions.
The answer is lack of [statistical power](https://en.wikipedia.org/wiki/Power_(statistics)), due to the fact that we only used 5 samples per model. 
We can see the difference if we repead the same procedure with 100 samples per model. (In a real work you should NOT adjust the number of samples based on the results without repeating your experiments).

In [40]:
results_many_samples = []
for split_idx in range(100):
    results_many_samples.append({"Model": "B1", "Split": split_idx, "MSE": np.random.normal(loc=0.87, scale=0.021)})
    results_many_samples.append({"Model": "B2", "Split": split_idx, "MSE": np.random.normal(loc=0.85, scale=0.034)})
    results_many_samples.append({"Model": "M", "Split": split_idx, "MSE": np.random.normal(loc=0.79, scale=0.10)})
results_many_samples = pd.DataFrame(results_many_samples)


best_model_samples = results_many_samples[results_many_samples["Model"]=="M"]["MSE"].tolist()

for alternative_model in ["B1", "B2"]:
    alternative_model_samples = results_many_samples[results_many_samples["Model"]==alternative_model]["MSE"].tolist()
    ttest_result = scipy.stats.ttest_ind(best_model_samples, alternative_model_samples, equal_var=False)
    if ttest_result.pvalue<statistical_significance_threshold:
        print(f"Difference between model M and model {alternative_model} is statistically significant: pvalue {ttest_result.pvalue}")
    else:
        print(f"The difference between model M and model {alternative_model} is NOT statistically significant: pvalue {ttest_result.pvalue}")


Difference between model M and model B1 is statistically significant: pvalue 6.9385199674497656e-09
Difference between model M and model B2 is statistically significant: pvalue 0.00022120169695229422
